# Lab | Customer Analysis Round 6

For this lab, we still keep using the marketing_customer_analysis.csv file that you can find in the files_for_lab folder.

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Get the data

We are using the marketing_customer_analysis.csv file.

In [34]:
df = pd.read_csv('marketing_customer_analysis.csv')
df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [35]:
# Change headers names

df.columns = df.columns.str.replace(' ', '_').str.lower()

df.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


## Dealing with the data

Already done in the round 2.

## Explore the data

Done in the round 3.

## Processing Data

(Further processing...)

- X-y split. (done)
- Normalize (numerical). (done)
- One Hot/Label Encoding (categorical).
- Concat DataFrames

In [36]:
#dealing with outliers

#create a list with the name of the columns where we detect outliers

outliers = ['customer_lifetime_value', 'monthly_premium_auto', 'total_claim_amount', 'number_of_policies']
def clean_outliers(data, outliers):

    for column in outliers:

        iqr = np.percentile(data[column],75) - np.percentile(data[column],25)

        upper_limit = np.percentile(data[column],75) + 1.5*iqr

        lower_limit = np.percentile(data[column],25) - 1.5*iqr

        data.loc[data[column] > upper_limit, column] = upper_limit

        data.loc[data[column] < lower_limit, column] = lower_limit

    return data

In [37]:
df2=clean_outliers(df,outliers)
df2.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1.0,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8.0,Personal Auto,Personal L3,Offer3,Agent,960.399730,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2.0,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7.0,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1.0,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


**X-Y Split** If you have not done it, you have you take in count that the target will be `total_claim_amount`

In [38]:
X=df2.drop('total_claim_amount', axis=1)
y=df2['total_claim_amount']

**Normalize (numerical)** If you have not done it yet, you can define a function using `StandardScaler`from sklearn library

In [39]:
# normalize only numerical columns
# get the dataframe as well as the list of columns

numerical = X.select_dtypes(np.number)


In [44]:
#normalization

transformer = Normalizer()
transformer.fit(numerical)

# saving in a pickle
with open('std_transformer.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer.pickle', 'rb') as file:
    loaded_normalizer = pickle.load(file)

X_normalized = loaded_normalizer.transform(numerical)

#Getting the normalized variables into a dataframe.

X_n = pd.DataFrame(X_normalized, columns = num_col)

**One Hot/Label Encoding (categorical)** Try one of the two options learned in class

In [42]:
df_cat = X.select_dtypes('object')

X_encoded = pd.get_dummies(df_cat, drop_first=True)


**Concat DataFrames**

In [45]:
X_final= pd.concat([X_n.reset_index(drop=True), X_encoded.reset_index(drop=True)], axis=1, ignore_index=True)

## Linear Regression

- Train-test split.
- Apply linear regression.

**Train-test split** Divide your data in a train part and a test part

In [46]:
# train-test split

X_train, X_test, y_train, y_test = train_test_split(X_final, y, random_state=0)

**Apply linear regression** For this question you can use `statsmodels` or `sklearn` libraries

In [47]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

## Model Validation

- Description:
R2.
MSE.
RMSE.
MAE.

**Get R2 from the model**

In [48]:
predictions = model.predict(X_test)

**Get MSE from the model**

In [49]:
mean_squared_error(y_test, predictions, squared=False)

111.9170047444073

**Get RMSE from the model**

In [50]:
r2_score(y_test, predictions)

0.7838800131896517

**Get MAE from the model**

In [53]:
 mean_absolute_error(y_test, predictions)

84.80100880592184